Slices of 5 frames, each slice will be individually delaunay-ed and then stacked on top of the previous one

In [1]:
import vtk
import numpy as np

In [2]:
# X = np.load('MasksForPlot.npy')
X = np.load('MasksNPYFiles/masksFinalContoured.npy')
# Y = X[:,:,:,0]
# Y = X[0:5,:,:,0]
Y = X[0:5,:,:]          #Set frames here
pts = np.argwhere(Y*1)
print(X.shape, X.size, Y.shape, Y.size)
print(len(pts.tolist()))
print(pts.tolist()[0:5])

(449, 384, 384) 66207744 (5, 384, 384) 737280
181
[[0, 232, 168], [0, 232, 171], [0, 232, 178], [0, 235, 164], [0, 236, 183]]


In [3]:
# def gen_points_sample():
    
#     pt = list()
    
#     for i in range(3,8):
#         for j in range(3,8):
#             for k in range(3,8):
#                 pt.append([i, j, k])
#     return pt

In [4]:
def gen_points_sample(X, i):
    
    pts = np.argwhere(X*1)
    points = pts.tolist()
#     npts = list()
#     for point in pt:
# #         point.reverse()
#         npts.append([point[2], point[1], point[0]])
    for pt in points:
        pt[0] = pt[0] + i
    return points

In [5]:
# points = vtk.vtkPoints()
# print(len(gen_points_sample()))
# for point in gen_points_sample():
#     points.InsertNextPoint(point)

In [6]:
# points = vtk.vtkPoints()
# print(len(gen_points_sample(Y)))
# for point in gen_points_sample(Y):
#     points.InsertNextPoint(point)

In [7]:
def get_vtkPoints_instance(pointsList):
    points = vtk.vtkPoints()
    for point in pointsList:
        points.InsertNextPoint(point)
    return points

In [8]:
# # Create a list of custom points
# points = vtk.vtkPoints()
# points.InsertNextPoint(0, 0, 0)
# points.InsertNextPoint(0, 3, 0)
# points.InsertNextPoint(0, 3, 13)
# points.InsertNextPoint(0, 3, 6)
# points.InsertNextPoint(7, 0, 0)
# points.InsertNextPoint(7, 0, 13)
# points.InsertNextPoint(7, 3, 13)
# points.InsertNextPoint(7, 3, 0)

In [9]:
# Create an unstructured grid from the points
# grid = vtk.vtkUnstructuredGrid()
# grid.SetPoints(points)

In [10]:
# Use the GPU tessellator to create a Delaunay triangulation
# tessellator = vtkGPUUnstructuredGridTessellator()
# tessellator.SetInputData(grid)
# tessellator.Update()

In [11]:
# # Create a point source and set its parameters
# point_source = vtk.vtkPointSource()
# point_source.SetNumberOfPoints(points.GetNumberOfPoints())
# point_source.SetCenter(0, 0, 0)
# point_source.SetRadius(1.0)

In [12]:
# # Set the custom points as the output of the point source
# point_source.Update()
# point_source.GetOutput().GetPoints().DeepCopy(points)

In [13]:
def build_vtkPointSource_from_vtkPoints(vtkPoints):
    point_source = vtk.vtkPointSource()
    point_source.SetNumberOfPoints(vtkPoints.GetNumberOfPoints())
    point_source.SetCenter(0, 0, 0)
    point_source.SetRadius(1.0)
    point_source.Update()
    point_source.GetOutput().GetPoints().DeepCopy(vtkPoints)
    return point_source

In [14]:
# # Use a Delaunay3D filter to generate a mesh from the points
# delaunay = vtk.vtkDelaunay3D()
# delaunay.SetInputData(point_source.GetOutput())
# delaunay.Update()

In [15]:
# # Convert to a geometric filter
# geom_filter = vtk.vtkGeometryFilter()
# geom_filter.SetInputConnection(delaunay.GetOutputPort())
# geom_filter.Update()

In [16]:
def getDelaunayFromPointSource(point_source):
    delaunay = vtk.vtkDelaunay3D()
    delaunay.SetInputData(point_source.GetOutput())
    delaunay.Update()
    return delaunay

In [17]:
def getGeomFilterFromDelaunay(delaunay):
    geom_filter = vtk.vtkGeometryFilter()
    geom_filter.SetInputConnection(delaunay.GetOutputPort())
    geom_filter.Update()
    return geom_filter

In [18]:
# # Create a mapper and actor for the mesh
# mapper = vtk.vtkPolyDataMapper()
# # mapper.SetInputData(point_source.GetOutput())
# mapper.SetInputData(geom_filter.GetOutput())
# actor = vtk.vtkActor()
# actor.SetMapper(mapper)

In [19]:
def buildActor(geom_filter):
    # Create a mapper and actor for the mesh
    mapper = vtk.vtkPolyDataMapper()
    # mapper.SetInputData(point_source.GetOutput())
    mapper.SetInputData(geom_filter.GetOutput())
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    return actor

In [20]:
# Z = X[0:5]
# pointsList = gen_points_sample(Z, 0)
# vtkPoints = get_vtkPoints_instance(pointsList)
# vtkPS = build_vtkPointSource_from_vtkPoints(vtkPoints)
# delaunay = getDelaunayFromPointSource(vtkPS)
# gf = getGeomFilterFromDelaunay(delaunay)
# actor = buildActor(gf)

# Z = X[350:355]
# pointsList = gen_points_sample(Z, 350)
# vtkPoints = get_vtkPoints_instance(pointsList)
# vtkPS = build_vtkPointSource_from_vtkPoints(vtkPoints)
# delaunay = getDelaunayFromPointSource(vtkPS)
# gf = getGeomFilterFromDelaunay(delaunay)
# actor1 = buildActor(gf)

In [21]:
if len([]):
    print("OK")

In [22]:
slice_len = 2
renderer = vtk.vtkRenderer()
print(X.shape[0] // slice_len)
print(X.shape[0])
for i in range(X.shape[0] // slice_len + 1):
# for i in range(86):
    k = min(X.shape[0] - 1, slice_len*(i+1))
    Z = X[slice_len*i:k]
    pointsList = gen_points_sample(Z, slice_len*i)
#     print(i)
    if not len(pointsList):
        print("No points at index ", i)
    else:
        vtkPoints = get_vtkPoints_instance(pointsList)
        vtkPS = build_vtkPointSource_from_vtkPoints(vtkPoints)
        delaunay = getDelaunayFromPointSource(vtkPS)
        gf = getGeomFilterFromDelaunay(delaunay)
        actor = buildActor(gf)
        renderer.AddActor(actor)

224
449
No points at index  212
No points at index  213
No points at index  214
No points at index  215
No points at index  216
No points at index  217
No points at index  224


In [23]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [24]:
# Create a renderer and add the actor to it
# renderer = vtk.vtkRenderer()
# renderer.AddActor(actor)
# renderer.AddActor(actor1)

In [25]:
# Create a render window and set its size
render_window = vtk.vtkRenderWindow()
render_window.SetSize(600, 600)

In [26]:
# Add the renderer to the render window
render_window.AddRenderer(renderer)

# Create an interactor and start the render loop
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)
interactor.Initialize()
interactor.Start()